<a href="https://colab.research.google.com/github/varmams2023/phd/blob/main/WorkingExtensionWithPCGANBlockChainWithCheques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colorama  # Install the colorama library

import hashlib
import json
import pandas as pd
from time import time
from typing import List
from colorama import Fore, Style, init
from tqdm import tqdm  # For progress bar
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim

# ---- Section 1: Install & Initialize Libraries ----
drive.mount('/content/drive')
init(autoreset=True)

# ---- Section 2: Block Class ----
class Block:
    """Class to represent each block in the blockchain."""

    def __init__(self, index, timestamp, cheque_data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.cheque_data = cheque_data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """Calculate the hash of the current block."""
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

# Debug: Block Class Initialized
print("Block Class Initialized.")

# ---- Section 3: Blockchain Class ----
class Blockchain:
    """Class to handle blockchain operations."""

    def __init__(self):
        self.chain = []
        self.pending_cheques = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Create the first block (genesis block)."""
        genesis_block = Block(0, time(), {"check_number": "0", "payee_name": "Genesis", "bank_name": "Genesis Bank", "account_number": "000000", "ifsc_code": "000000"}, "0")
        self.chain.append(genesis_block)

    def add_cheque(self, cheque_data):
        """Add cheque data as a new block to the blockchain."""
        previous_hash = self.chain[-1].hash
        new_block = Block(len(self.chain), time(), cheque_data, previous_hash)
        self.chain.append(new_block)
        self.pending_cheques = []  # Reset pending cheques after adding

    def add_cheques_to_blockchain(self, cheques: List[dict]):
        """Method to add multiple cheques to the blockchain."""
        for cheque in tqdm(cheques, desc="Adding Cheques to Blockchain", unit="cheque"):
            self.add_cheque(cheque)

    def validate_cheque(self, cheque_data):
        """Validate if the given cheque data exists in the blockchain."""
        for block in self.chain:
            if block.cheque_data == cheque_data:
                return True  # Valid cheque data found
        return False

    def display_chain(self, num_blocks=15):
        """Display a limited number of blocks from the blockchain."""
        print(f"\nDisplaying first {num_blocks} blocks from the blockchain:\n")
        for block in self.chain[:num_blocks]:
            print(f"\n{Fore.GREEN}Block #{block.index} ({Fore.CYAN}{block.timestamp}{Style.RESET_ALL})")
            print(f"{Fore.YELLOW}Check Number: {block.cheque_data['check_number']}")
            print(f"{Fore.YELLOW}Payee Name: {block.cheque_data['payee_name']}")
            print(f"{Fore.YELLOW}Bank Name: {block.cheque_data['bank_name']}")
            print(f"{Fore.YELLOW}Account Number: {block.cheque_data['account_number']}")
            print(f"{Fore.YELLOW}IFSC Code: {block.cheque_data['ifsc_code']}")
            print(f"{Fore.BLUE}Hash: {block.hash}\n")

    def count_blocks_by_bank(self):
        """Count the number of blocks grouped by bank."""
        bank_count = {}
        for block in self.chain:
            bank_name = block.cheque_data['bank_name']
            if bank_name in bank_count:
                bank_count[bank_name] += 1
            else:
                bank_count[bank_name] = 1
        return bank_count

    def total_blocks(self):
        """Return the total number of blocks in the blockchain."""
        return len(self.chain)

# Debug: Blockchain Class Initialized
print("Blockchain Class Initialized.")

# ---- Section 4: Loading Cheque Data from CSV ----
def load_cheque_data_from_csv(file_path: str):
    """Load cheque data from a CSV file."""
    df = pd.read_csv(file_path)

    # Ensure only relevant columns are selected
    cheque_data = df[['CheckNumber', 'AccountNumber', 'PayeeName', 'BankName', 'IFSCCode']]

    # Convert the DataFrame into a list of dictionaries
    cheque_data_list = cheque_data.rename(columns={
        'CheckNumber': 'check_number',
        'AccountNumber': 'account_number',
        'PayeeName': 'payee_name',
        'BankName': 'bank_name',
        'IFSCCode': 'ifsc_code'
    }).to_dict(orient='records')

    return cheque_data_list

# File path for the CSV in Google Drive
file_path = '/content/drive/MyDrive/phd/dataset/ChequeDataSetFull.csv'

# Load cheque data from CSV
cheque_data_list = load_cheque_data_from_csv(file_path)

# Debug: Data Loaded from CSV
print(f"Loaded {len(cheque_data_list)} cheque records from CSV.")

# ---- Section 5: Fake Check Detection with PCGAN ----
class FakeCheckPCGAN(nn.Module):
    """A simple Progressive Conditional GAN (PCGAN) for fake cheque detection."""

    def __init__(self):
        super(FakeCheckPCGAN, self).__init__()
        self.generator = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 2048),
            nn.Tanh()  # Assuming cheque data is normalized between -1 and 1
        )
        self.discriminator = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.Sigmoid()  # Output for fake/real classification
        )

    def forward(self, z):
        generated_data = self.generator(z)
        validity = self.discriminator(generated_data)
        return validity

    def train(self, real_data, fake_data):
        optimizer = optim.Adam(self.parameters(), lr=0.0002, betas=(0.5, 0.999))
        criterion = nn.BCELoss()

        # Train discriminator
        real_labels = torch.ones(real_data.size(0), 1)
        fake_labels = torch.zeros(fake_data.size(0), 1)

        # Forward pass for real and fake data
        real_output = self.discriminator(real_data)
        fake_output = self.discriminator(fake_data)

        # Loss calculation
        real_loss = criterion(real_output, real_labels)
        fake_loss = criterion(fake_output, fake_labels)
        d_loss = real_loss + fake_loss

        optimizer.zero_grad()
        d_loss.backward()
        optimizer.step()

        # Train generator
        z = torch.randn(real_data.size(0), 100)  # Latent vector
        generated_data = self.generator(z)
        validity = self.discriminator(generated_data)

        g_loss = criterion(validity, real_labels)  # We want the generator to fool the discriminator

        optimizer.zero_grad()
        g_loss.backward()
        optimizer.step()

        return d_loss, g_loss

# Debug: Fake Check PCGAN Model Initialized
print("PCGAN Model Initialized for Fake Check Detection.")

# ---- Section 6: Initialize Blockchain ----
blockchain = Blockchain()
blockchain.add_cheques_to_blockchain(cheque_data_list)

# Debug: Blockchain Initialized and Cheques Added
print(f"Blockchain initialized with {len(blockchain.chain)} blocks.")

# ---- Section 7: Menu for User Input ----
def display_menu():
    """Display the menu and handle user input."""
    while True:
        print("\nMenu:")
        print("1. Print Blockchain")
        print("2. Add Block (Manually input cheque data)")
        print("3. Validate if Cheque is present")
        print("4. Print Count of Blocks by Bank")
        print("5. Exit")
        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '1':
            # Take number of blocks to print as input
            num_blocks = int(input("Enter the number of blocks to display: "))
            blockchain.display_chain(num_blocks=num_blocks)

        elif choice == '2':
            # Manually input the fields for a new block
            check_number = input("Enter Check Number: ")
            payee_name = input("Enter Payee Name: ")
            bank_name = input("Enter Bank Name: ")
            account_number = input("Enter Account Number: ")
            ifsc_code = input("Enter IFSC Code: ")

            cheque_data = {
                'check_number': check_number,
                'payee_name': payee_name,
                'bank_name': bank_name,
                'account_number': account_number,
                'ifsc_code': ifsc_code
            }
            blockchain.add_cheque(cheque_data)
            print(Fore.GREEN + "Block added successfully!")

        elif choice == '3':
            # Manually input the fields for cheque validation
            check_number = input("Enter Check Number: ")
            payee_name = input("Enter Payee Name: ")
            bank_name = input("Enter Bank Name: ")
            account_number = input("Enter Account Number: ")
            ifsc_code = input("Enter IFSC Code: ")

            cheque_data = {
                'check_number': check_number,
                'payee_name': payee_name,
                'bank_name': bank_name,
                'account_number': account_number,
                'ifsc_code': ifsc_code
            }
            is_valid = blockchain.validate_cheque(cheque_data)
            if is_valid:
                print(Fore.GREEN + "Cheque is present in the blockchain.")
            else:
                print(Fore.RED + "Cheque is NOT present in the blockchain.")

        elif choice == '4':
            # Print count of blocks grouped by bank and total block count
            bank_count = blockchain.count_blocks_by_bank()
            total_blocks = blockchain.total_blocks()
            print(f"\nTotal Blocks in Blockchain: {total_blocks}")
            print("\nCount of Blocks by Bank:")
            for bank, count in bank_count.items():
                print(f"{Fore.YELLOW}{bank}: {count} blocks")

        elif choice == '5':
            print("Exiting...")
            break
        else:
            print(Fore.RED + "Invalid choice. Please try again.")

# Debug: Menu Initialized
print("Menu system initialized.")

# Start the menu
display_menu()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Block Class Initialized.
Blockchain Class Initialized.
Loaded 100000 cheque records from CSV.
PCGAN Model Initialized for Fake Check Detection.


Adding Cheques to Blockchain: 100%|██████████| 100000/100000 [00:01<00:00, 58847.77cheque/s]


Blockchain initialized with 100001 blocks.
Menu system initialized.

Menu:
1. Print Blockchain
2. Add Block (Manually input cheque data)
3. Validate if Cheque is present
4. Print Count of Blocks by Bank
5. Exit
Enter your choice (1/2/3/4/5): 1
Enter the number of blocks to display: 5

Displaying first 5 blocks from the blockchain:


Block #0 (1734710135.0483603)
Check Number: 0
Payee Name: Genesis
Bank Name: Genesis Bank
Account Number: 000000
IFSC Code: 000000
Hash: 9fddd17e73122ef5f533edc6a497d2abd504d643fed6798a839e074c2a42148c


Block #1 (1734710135.0571945)
Check Number: 65782867
Payee Name: Devika Reddy
Bank Name: State Bank of India
Account Number: 3156841378
IFSC Code: SBIN0008572
Hash: 10646c22d2bef2ff250caaabefc9b0a08763821c0931b0d4c3a827466846eb3e


Block #2 (1734710135.0572884)
Check Number: 56855036
Payee Name: Aanya Sharma
Bank Name: State Bank of India
Account Number: 7130589870
IFSC Code: SBIN0008316
Hash: 5c7b22a5a76f6b6861dd4c4d58ef869c1e48e4f138ab233dc5b96f5830d56bbb